In [1]:
import pandas as pd
import pickle

from collections import defaultdict
import string

import torch
from torch import nn
from torch.nn.functional import softmax
from torch.utils.data import DataLoader
from transformers import DistilBertConfig, DistilBertTokenizer
from transformers import RobertaConfig
from datasets import Dataset
from transformers import DataCollatorWithPadding
from torch.utils.data import DataLoader

from IPython.display import display, HTML

from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding

2025-07-12 15:10:08.727455: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752333008.916668      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752333008.972400      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
import torch
import torch.nn as nn
from transformers import AutoModelForSequenceClassification
from transformers.models.gemma3.configuration_gemma3 import (
    Gemma3Config, Gemma3TextConfig,
)
from transformers.models.gemma3.modeling_gemma3 import (
    Gemma3TextModel, Gemma3PreTrainedModel,
)
from transformers.modeling_outputs import SequenceClassifierOutputWithPast


# ───────────────────── helpers ─────────────────────────────────────────
def _txt(cfg):                              # text sub-config
    return getattr(cfg, "text_config", cfg)


def _pool_last(token_logits, input_ids, pad_id):
    if input_ids is None or pad_id is None:
        return token_logits[:, -1]
    ends = (~input_ids.eq(pad_id)).cumsum(-1).argmax(-1)
    return token_logits[torch.arange(token_logits.size(0)), ends]


def _compute_loss(config, logits, labels, num_labels):
    if labels is None:
        return None
    if config.problem_type is None:
        if num_labels == 1:
            config.problem_type = "regression"
        elif labels.dtype in (torch.long, torch.int):
            config.problem_type = "single_label_classification"
        else:
            config.problem_type = "multi_label_classification"

    if config.problem_type == "regression":
        return nn.MSELoss()(logits.squeeze(), labels.squeeze())
    if config.problem_type == "single_label_classification":
        return nn.CrossEntropyLoss()(logits, labels)
    return nn.BCEWithLogitsLoss()(logits, labels)

# ───────────────────── text-only variant (1 B) ────────────────────────
class Gemma3TextForSequenceClassification(Gemma3PreTrainedModel):
    """
    **Wraps** Gemma3TextModel in `self.model` – keeps `model.*`
    prefixes so every pretrained weight loads.
    """
    config_class = Gemma3TextConfig
    keys_to_ignore_at_inference = ["past_key_values"]

    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.model = Gemma3TextModel(config)              # <- wrapper
        self.score = nn.Linear(config.hidden_size,
                               self.num_labels, bias=False)
        self.post_init()

    def forward(
        self,
        input_ids=None, attention_mask=None, position_ids=None,
        inputs_embeds=None, past_key_values=None, labels=None,
        use_cache=None, output_attentions=True,
        output_hidden_states=None, return_dict=None,
    ):
        return_dict = (return_dict if return_dict is not None
                       else self.config.use_return_dict)

        outputs = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            position_ids=position_ids,
            inputs_embeds=inputs_embeds,
            past_key_values=past_key_values,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=True,
        )
        token_logits = self.score(outputs.last_hidden_state)
        pooled = _pool_last(token_logits, input_ids, self.config.pad_token_id)
        loss = _compute_loss(self.config, pooled, labels, self.num_labels)

        if not return_dict:
            out = (pooled,) + outputs[1:]
            return ((loss,) + out) if loss is not None else out

        return SequenceClassifierOutputWithPast(
            loss=loss, logits=pooled,
            past_key_values=outputs.past_key_values,
            hidden_states=outputs.hidden_states
                if output_hidden_states else None,
            attentions=outputs.attentions
                if output_attentions else None,
        )


# ───────────────────── register with HF factory ───────────────────────
AutoModelForSequenceClassification.register(
    Gemma3TextConfig, Gemma3TextForSequenceClassification)

In [ ]:
from huggingface_hub import login

login(new_session=False, # Won’t request token if one is already saved on machine
write_permission=True, # Requires a token with write permission
token="", # The name of your token
add_to_git_credential=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in 'login': write_permission. Will not be supported from version '1.0'.

Fine-grained tokens added complexity to the permissions, making it irrelevant to check if a token has 'write' access.
  warnings.warn(message, FutureWarning)
Token has not been saved to git credential helper.


Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.


In [4]:
checkpoint = "ConditionalNLI/Exp8_gemma3"

In [5]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3, device_map="auto", output_attentions=True)

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/822 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/899 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

Some weights of Gemma3TextForSequenceClassification were not initialized from the model checkpoint at google/gemma-3-1b-it and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


adapter_model.safetensors:   0%|          | 0.00/13.1M [00:00<?, ?B/s]

## Running on Dataset

In [7]:
from datasets import load_dataset, concatenate_datasets
import json
import ast

In [8]:
test_df4 = pd.read_excel('/kaggle/input/confer-extension/Part5_Dataset/Part5_Type4.xlsx')
test_df4

,premise,hypothesis,gold_label,trigger,key_phrase1,key_phrase2,type
0,"If Liam is an aerospace engineer, he'll use hi...",Liam has a wind tunnel.,E,possessive,his wind tunnel,improve aircraft performance,related
1,"If Bill is a content strategist, he'll use his...",Bill has a content management application.,E,possessive,his content management application,plan and organize digital content,related
2,"If Steve is a conservation officer, he'll use ...",Steve has a wildlife trap.,E,possessive,his wildlife trap,monitor and protect animal populations,related
3,"If Matt is a scuba diver, he'll wear his wetsu...",Matt has a wetsuit.,E,possessive,his wetsuit,maintain body temperature underwater,related
4,"If Noah is a pastry chef, he'll use his pastry...",Noah has a pastry bag.,E,possessive,his pastry bag,decorate cakes and pastries,related
...,...,...,...,...,...,...,...
292,"If Christina is a makeup artist, she'll grab h...",Christina has a makeup sponge.,E,possessive,her makeup sponge,rides a scooter,unrelated
293,"If Rahim is a clergyman, he'll bring his holy ...",Rahim has a holy book.,E,possessive,his holy book,gardens roses,unrelated
294,"If James is a police officer, he'll pick his g...",James has a gun.,E,possessive,his gun,drinks tea,unrelated
295,"If Maya is a piano teacher, she'll pack her me...",Maya has a metronome.,E,possessive,her metronome,knits hats,unrelated


In [9]:
test_df5p = pd.read_excel('/kaggle/input/confer-extension/Part5_Dataset/Part5_Type5P.xlsx')
test_df5p

,premise,hypothesis,gold_label,trigger,key_phrase1,key_phrase2,type
0,"If Jessica attends the conference, her colleag...",Jessica has a colleague.,E,possessive,her colleague,the university,related
1,"If Emily passes her driving test, her best fri...",Emily has a best friend.,E,possessive,her best friend,post exam celebratory dinner,related
2,"If Scarlett finishes her chores early, her hus...",Scarlett has a husband.,E,possessive,her husband,lunch,related
3,"If Chloe finishes his project early, his colle...",Chloe has colleagues.,E,possessive,his colleagues,drinks,related
4,"If Charlotte passes her math test, her parents...",Charlotte has parents.,E,possessive,her parents,a calculator,related
...,...,...,...,...,...,...,...
795,"If Dorothy bike breaks down, her brother will ...",Dorothy has a brother.,E,possessive,her brother,water her garden at home,unrelated
796,"If Susan finishes her assignment early, her co...",Susan has a colleague.,E,possessive,his colleague,dust his shelves at the dorm,unrelated
797,"If Margaret finishes work early, her wife will...",Margaret has a wife.,E,possessive,his wife,plant a tree for him,unrelated
798,"If Betty misses the train, her brother will ma...",Betty has a brother.,E,possessive,her brother,make her origami to class,unrelated


In [10]:
test_df5a = pd.read_excel('/kaggle/input/confer-extension/Part5_Dataset/Part5_Type5A.xlsx')
test_df5a

,premise,hypothesis,gold_label,trigger,key_phrase1,key_phrase2,type
0,"If Jessica attends a movie festival, she'll ne...",Jessica has watched a movie by Nolan before.,E,again,attends a movie festival,watch a movie by Nolan again,related
1,"If Emily finishes a mystery novel, she'll neve...",Emily has read a book by Dan Brown before.,E,again,finishes a mystery novel,read a book by Dan Brown again,related
2,"If Scarlett bakes a Swedish pastry, she'll nev...",Scarlett has eaten a traditional Swedish dish ...,E,again,bakes a Swedish pastry,eat a traditional Swedish dish again,related
3,"If Chloe attends a pop concert, she'll never s...",Chloe has sung a pop song before.,E,again,attends a pop concert,sing a pop song again,related
4,"If Charlotte attends a pop music festival, she...",Charlotte has listened to a pop song before.,E,again,attends a pop music festival,listen to a pop song again,related
...,...,...,...,...,...,...,...
795,"If Dorothy's car breaks down, she'll never obt...",Dorothy has obtained a job position before.,E,again,breaks down,obtain a job position again,unrelated
796,"If Susan finishes her work early, she'll never...",Susan has conveyed a confidential message before.,E,again,finishes her work early,convey a confidential message again,unrelated
797,"If Margaret finishes work early, she'll never ...",Margaret has managed to keep a secret before.,E,again,finishes work early,manage to keep a secret again,unrelated
798,"If Betty misses the bus, she'll never cherish ...",Betty has cherished a moment in peace before.,E,again,misses the bus,cherish a moment in peace again,unrelated


In [11]:
test_df = pd.concat([test_df4,test_df5a,test_df5p])
test_df

,premise,hypothesis,gold_label,trigger,key_phrase1,key_phrase2,type
0,"If Liam is an aerospace engineer, he'll use hi...",Liam has a wind tunnel.,E,possessive,his wind tunnel,improve aircraft performance,related
1,"If Bill is a content strategist, he'll use his...",Bill has a content management application.,E,possessive,his content management application,plan and organize digital content,related
2,"If Steve is a conservation officer, he'll use ...",Steve has a wildlife trap.,E,possessive,his wildlife trap,monitor and protect animal populations,related
3,"If Matt is a scuba diver, he'll wear his wetsu...",Matt has a wetsuit.,E,possessive,his wetsuit,maintain body temperature underwater,related
4,"If Noah is a pastry chef, he'll use his pastry...",Noah has a pastry bag.,E,possessive,his pastry bag,decorate cakes and pastries,related
...,...,...,...,...,...,...,...
795,"If Dorothy bike breaks down, her brother will ...",Dorothy has a brother.,E,possessive,her brother,water her garden at home,unrelated
796,"If Susan finishes her assignment early, her co...",Susan has a colleague.,E,possessive,his colleague,dust his shelves at the dorm,unrelated
797,"If Margaret finishes work early, her wife will...",Margaret has a wife.,E,possessive,his wife,plant a tree for him,unrelated
798,"If Betty misses the train, her brother will ma...",Betty has a brother.,E,possessive,her brother,make her origami to class,unrelated


In [12]:
test_df.to_csv('part5_all.csv')

In [16]:
def tokenize_function(example):
    return tokenizer(example["premise"], example["hypothesis"], truncation=True, max_length=256, padding='max_length')

In [17]:
def predict_with_probs(test_df):

    test_ds = Dataset.from_pandas(test_df)
    columns_to_keep = ['index', 'premise', 'hypothesis']
    test_ds = test_ds.remove_columns([col for col in test_ds.column_names if col not in columns_to_keep])
    tokenized_test_ds = test_ds.map(tokenize_function, batched=True)
    tokenized_test_ds.set_format("torch")
    tokenized_test_ds = tokenized_test_ds.remove_columns(["premise", "hypothesis"])
    
    # Predicting Labels on Dataset
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding="max_length", max_length=256, return_tensors="pt")
    dataloader = DataLoader(tokenized_test_ds, batch_size=32, collate_fn=data_collator)
    
    model.eval()
    predictions = []
    probabilities = []
    
    with torch.no_grad():
        for batch in dataloader:
            batch = {k: v.to("cuda") for k, v in batch.items()}
            outputs = model(**batch)
            logits = outputs.logits
            probs = softmax(logits, dim=1)
            preds = torch.argmax(probs, dim=1)
            
            predictions.extend(preds.cpu().tolist())
            probabilities.extend(probs.cpu().tolist())
    
    test_df['predicted_label'] = predictions
    test_df['prediction_probs'] = probabilities
    return test_df
    

In [18]:
import string

def is_content_token(token):
    # Remove special tokens 
    return token not in tokenizer.all_special_tokens

In [19]:
def compute_attention(row):

    premise = row['premise']
    hypothesis = row['hypothesis']
    inputs = tokenizer(premise, hypothesis, return_tensors='pt')
    inputs.pop('token_type_ids', None)  # for llama 

    device = next(model.parameters()).device
    inputs = {k: v.to(device) for k, v in inputs.items()} 
    
    with torch.no_grad():
        outputs = model(**inputs)
        attentions = outputs.attentions

    return inputs, attentions

In [21]:
def cls_token_attention(row):
    # Aggregating Attention scores 
    attn_tensor = torch.stack(outputs.attentions)  # shape: (num_layers, batch, num_heads, seq_len, seq_len)
    cls_index = 0  # RoBERTa’s <s> token is at index 0
    cls_attention = attn_tensor[:, :, :, cls_index, :]  # shape: (num_layers, batch, num_heads, seq_len)

    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    attention_values = cls_attention.mean(dim=(0,1,2)).squeeze().tolist()
    
    # Filter
    filtered_tokens = []
    filtered_attention = []
    
    for token, attn in zip(tokens, attention_values):
        if is_content_token(token):
            filtered_tokens.append(token)
            filtered_attention.append(attn)

    plt.figure(figsize=(12, 6))
    plt.bar(range(len(filtered_tokens)), filtered_attention)
    plt.xticks(range(len(filtered_tokens)), filtered_tokens, rotation=90)
    plt.title("Attention from <s> Token (Filtered to Words Only)")
    plt.tight_layout()
    plt.show()

## Average Attention

In [22]:
import matplotlib.pyplot as plt
import seaborn as sns

In [29]:
def average_attention(row): 

    inputs, attention = compute_attention(row)
    
    attn_tensor = torch.stack(attention)  # Shape: (num_layers, batch, num_heads, seq_len, seq_len)
    avg_attn = attn_tensor.mean(dim=(0, 1, 2))  # Shape: (seq_len, seq_len)
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

    filtered_tokens = []
    filtered_attention = []

    for token, attn in zip(tokens, avg_attn.detach().cpu().numpy()):
        if is_content_token(token):
            filtered_tokens.append(token)
            filtered_attention.append(attn)

    plt.figure(figsize=(12, 10))
    sns.heatmap(avg_attn.detach().cpu().numpy(), xticklabels=tokens, yticklabels=tokens, cmap="viridis", annot=False, fmt=".2f", linewidths=0)
    
    plt.title("Average Attention Between Tokens")
    plt.xlabel("Attended Token")
    plt.ylabel("Attending Token")
    plt.xticks(rotation=90)
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.savefig("attention_heatmap.pdf", dpi=600, bbox_inches='tight')
    plt.show()

    avg_attn = avg_attn.detach().cpu().numpy()    

    return pd.Series([avg_attn, tokens])

In [24]:
sample_test_df

,premise,hypothesis,gold_label,trigger,key_phrase1,key_phrase2,type
0,"If Liam is an aerospace engineer, he'll use hi...",Liam has a wind tunnel.,E,possessive,his wind tunnel,improve aircraft performance,related
99,"If Liam is an aerospace engineer, he'll use hi...",Liam has a wind tunnel.,E,possessive,his wind tunnel,cool off on a hot summer day,somewhat_related
198,"If Liam is an aerospace engineer, he'll grab h...",Liam has a wind tunnel.,E,possessive,his wind tunnel,feeds the cat,unrelated


In [23]:
def is_partial_word(tok,prev_tok):
    return not tok.startswith('▁') \
                and tok not in string.punctuation \
                and tok not in tokenizer.all_special_tokens \
                and prev_tok not in tokenizer.all_special_tokens

In [94]:
def compute_attention_metrics(row):

    avg_attention = row['attention']
    tokens =  row['tokens']
    attention_df = pd.DataFrame(avg_attention,index=tokens,columns=tokens)

    # clean attention matrix 
    partial_words = [tok for i,tok in enumerate(attention_df.index) if not tok.startswith('▁') 
                and tok not in string.punctuation
                and tok not in tokenizer.all_special_tokens
                and attention_df.index[i-1] not in tokenizer.all_special_tokens]

    # if it's a partial word, add its row to the previous row (merging tokens)
    for i in range(len(attention_df)-1,0,-1):
        current_token = attention_df.index[i]
        prev_token = attention_df.index[i-1]
        if is_partial_word(current_token,prev_token):
            attention_df.iloc[i-1] += attention_df.iloc[i] # first row gets total value
            attention_df.iloc[i] = 0 # set second row to 0
            merged_token = attention_df.index.values[i-1] + attention_df.index.values[i]
            attention_df.index.values[i-1] = merged_token

    attention_df.index = [token[1:] if token.startswith('▁') else token for token in attention_df.index]

    # repeat for columns
    for i in range(attention_df.shape[1]-1,0,-1):
        current_token = attention_df.columns[i]
        prev_token = attention_df.columns[i-1]
        if is_partial_word(current_token,prev_token):
            attention_df.iloc[:, i-1] += attention_df.iloc[:, i]
            attention_df.iloc[:, i] = 0
            merged_token = attention_df.columns.values[i - 1] + current_token
            attention_df.columns.values[i-1] = merged_token

    attention_df.columns = [token[1:] if token.startswith('▁') else token for token in attention_df.columns ]

    # dropping rows and columns that aren't words
    attention_df = attention_df[attention_df.index.to_series().apply(is_content_token)]
    drop_cols = [col for col in attention_df.columns if not is_content_token(col)]
    attention_df = attention_df.drop(columns=drop_cols)
    # print(attention_df)

    # compute overall average attention
    avg_attention = attention_df.mean().mean()

    # compute average attention from key_phrase1 to key_phrase2 
    key_phrase1 = row['key_phrase1']
    key_phrase2 = row['key_phrase2']
    
    key_phrase1_words = key_phrase1.split()
    key_phrase2_words = key_phrase2.split()

    start_key_phrase1 = key_phrase1_words[0]
    end_key_phrase1 = key_phrase1_words[-1]
    start_key_phrase2 = key_phrase2_words[0]
    end_key_phrase2 = key_phrase2_words[-1]
    phrase_thresholds = [start_key_phrase1, end_key_phrase1, start_key_phrase2, end_key_phrase2]
    indices = {}
    try:
        indices['start_key_phrase1'] = list(attention_df.index).index(start_key_phrase1)
    except: 
        if 'his' in attention_df.index: 
            start_key_phrase1 = 'his'
        elif 'her' in attention_df.index: 
            start_key_phrase1 = 'her'
        elif 'their' in attention_df.index: 
            start_key_phrase1 = 'their'
        else:
            return pd.Series([avg_attention,None])

    try:
        indices['start_key_phrase2'] = list(attention_df.index).index(start_key_phrase2)
    except:
        if 'to' in attention_df.index: 
            indices['start_key_phrase2'] = list(attention_df.index).index('to') + 1
        else:
            return pd.Series([avg_attention,None])

    # searching for end index starts after start index
    try:
        indices['end_key_phrase1'] = indices['start_key_phrase1'] + list(attention_df.index)[indices['start_key_phrase1']:].index(end_key_phrase1) + 1
    except:
        if 'to' in attention_df.index: 
            indices['end_key_phrase1'] = list(attention_df.index).index('to')
        else:
            return pd.Series([avg_attention,None])

    try:
        indices['end_key_phrase2'] = indices['start_key_phrase2'] + list(attention_df.index)[indices['start_key_phrase2']:].index(end_key_phrase2) + 1
    except:
        if '.' in attention_df.index: 
            indices['end_key_phrase2'] = list(attention_df.index).index('.')
        else:
            return pd.Series([avg_attention,None])
    
    # print(indices)
    
    key_phrase_attention = attention_df.iloc[indices['start_key_phrase1']:indices['end_key_phrase1'],indices['start_key_phrase1']:indices['end_key_phrase1']]
    avg_key_phrase_attention = key_phrase_attention.mean().mean()
    # print(avg_key_phrase_attention)
    
    return pd.Series([avg_attention,avg_key_phrase_attention])

In [95]:
sample_test_df[['attention','tokens']] = sample_test_df.apply(average_attention,axis=1)

In [96]:
sample_test_df[['avg_attention','avg_key_phrase_attention']] = sample_test_df.apply(compute_attention_metrics, axis=1)
sample_test_df

,index,premise,hypothesis,gold_label,trigger,key_phrase1,key_phrase2,type,attention,tokens,avg_attention,avg_key_phrase_attention
0,0,"If Liam is an aerospace engineer, he'll use hi...",Liam has a wind tunnel.,E,possessive,his wind tunnel,improve aircraft performance,related,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁Liam, ▁is, ▁an, ▁aerospace, ▁engi...",0.019754,0.068244
1,1,"If Bill is a content strategist, he'll use his...",Bill has a content management application.,E,possessive,his content management application,plan and organize digital content,related,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁Bill, ▁is, ▁a, ▁content, ▁strateg...",0.017797,0.059054
2,2,"If Steve is a conservation officer, he'll use ...",Steve has a wildlife trap.,E,possessive,his wildlife trap,monitor and protect animal populations,related,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁Steve, ▁is, ▁a, ▁conservation, ▁o...",0.018578,0.071189
3,3,"If Matt is a scuba diver, he'll wear his wetsu...",Matt has a wetsuit.,E,possessive,his wetsuit,maintain body temperature underwater,related,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁Matt, ▁is, ▁a, ▁scuba, ▁diver, ,,...",0.019159,0.160187
4,4,"If Noah is a pastry chef, he'll use his pastry...",Noah has a pastry bag.,E,possessive,his pastry bag,decorate cakes and pastries,related,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁Noah, ▁is, ▁a, ▁pastry, ▁chef, ,,...",0.019311,0.067246
5,5,"If Donald is a crime scene investigator, he'll...",Donald has evidence bags.,E,possessive,his evidence bags,secure crime scene samples,related,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁Donald, ▁is, ▁a, ▁crime, ▁scene, ...",0.019588,0.067692
6,6,"If Edward is a playwright, he'll use his pen t...",Edward has a pen.,E,possessive,his pen,write scripts for the stage,related,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁Edward, ▁is, ▁a, ▁playwright, ,, ...",0.020798,0.073165
7,7,"If William is a notary, he'll use his notary s...",William has a notary seal.,E,possessive,his notary seal,certify documents,related,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁William, ▁is, ▁a, ▁notary, ,, ▁he...",0.021506,0.067795


In [86]:
len(test_df4)

297

In [97]:
test_df4[['attention','tokens']] = test_df4.apply(average_attention,axis=1)
test_df4[['avg_attention','avg_key_phrase_attention']] = test_df4.apply(compute_attention_metrics, axis=1)
test_df4

,premise,hypothesis,gold_label,trigger,key_phrase1,key_phrase2,type,attention,tokens,avg_attention,avg_key_phrase_attention
0,"If Liam is an aerospace engineer, he'll use hi...",Liam has a wind tunnel.,E,possessive,his wind tunnel,improve aircraft performance,related,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁Liam, ▁is, ▁an, ▁aerospace, ▁engi...",0.019754,0.068244
1,"If Bill is a content strategist, he'll use his...",Bill has a content management application.,E,possessive,his content management application,plan and organize digital content,related,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁Bill, ▁is, ▁a, ▁content, ▁strateg...",0.017797,0.059054
2,"If Steve is a conservation officer, he'll use ...",Steve has a wildlife trap.,E,possessive,his wildlife trap,monitor and protect animal populations,related,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁Steve, ▁is, ▁a, ▁conservation, ▁o...",0.018578,0.071189
3,"If Matt is a scuba diver, he'll wear his wetsu...",Matt has a wetsuit.,E,possessive,his wetsuit,maintain body temperature underwater,related,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁Matt, ▁is, ▁a, ▁scuba, ▁diver, ,,...",0.019159,0.160187
4,"If Noah is a pastry chef, he'll use his pastry...",Noah has a pastry bag.,E,possessive,his pastry bag,decorate cakes and pastries,related,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁Noah, ▁is, ▁a, ▁pastry, ▁chef, ,,...",0.019311,0.067246
...,...,...,...,...,...,...,...,...,...,...,...
292,"If Christina is a makeup artist, she'll grab h...",Christina has a makeup sponge.,E,possessive,her makeup sponge,rides a scooter,unrelated,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁Christina, ▁is, ▁a, ▁makeup, ▁art...",0.020029,0.061750
293,"If Rahim is a clergyman, he'll bring his holy ...",Rahim has a holy book.,E,possessive,his holy book,gardens roses,unrelated,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁Rahim, ▁is, ▁a, ▁clergyman, ,, ▁h...",0.020792,0.061892
294,"If James is a police officer, he'll pick his g...",James has a gun.,E,possessive,his gun,drinks tea,unrelated,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁James, ▁is, ▁a, ▁police, ▁officer...",0.021970,0.073059
295,"If Maya is a piano teacher, she'll pack her me...",Maya has a metronome.,E,possessive,her metronome,knits hats,unrelated,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁Maya, ▁is, ▁a, ▁piano, ▁teacher, ...",0.019871,0.137232


In [98]:
test_df4['avg_attention'].mean()

0.020083266696093057

In [99]:
test_df4['avg_key_phrase_attention'].mean()

0.07443494757486356

In [100]:
test_df4.groupby('type')['avg_key_phrase_attention'].mean()

type
related             0.075107
somewhat_related    0.072855
unrelated           0.075350
Name: avg_key_phrase_attention, dtype: float64

In [101]:
test_df4.to_csv('gemma_part5_type4.csv')

## Type 5P

In [102]:
test_df5p[['attention','tokens']] = test_df5p.apply(average_attention,axis=1)
test_df5p[['avg_attention','avg_key_phrase_attention']] = test_df5p.apply(compute_attention_metrics, axis=1)
test_df5p

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,premise,hypothesis,gold_label,trigger,key_phrase1,key_phrase2,type,attention,tokens,avg_attention,avg_key_phrase_attention
0,"If Jessica attends the conference, her colleag...",Jessica has a colleague.,E,possessive,her colleague,the university,related,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁Jessica, ▁attends, ▁the, ▁confere...",0.024926,0.085384
1,"If Emily passes her driving test, her best fri...",Emily has a best friend.,E,possessive,her best friend,post exam celebratory dinner,related,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁Emily, ▁passes, ▁her, ▁driving, ▁...",0.018650,0.046570
2,"If Scarlett finishes her chores early, her hus...",Scarlett has a husband.,E,possessive,her husband,lunch,related,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁Scarlett, ▁finishes, ▁her, ▁chore...",0.022118,0.050348
3,"If Chloe finishes his project early, his colle...",Chloe has colleagues.,E,possessive,his colleagues,drinks,related,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁Chloe, ▁finishes, ▁his, ▁project,...",0.024816,0.051502
4,"If Charlotte passes her math test, her parents...",Charlotte has parents.,E,possessive,her parents,a calculator,related,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁Charlotte, ▁passes, ▁her, ▁math, ...",0.025304,0.052705
...,...,...,...,...,...,...,...,...,...,...,...
795,"If Dorothy bike breaks down, her brother will ...",Dorothy has a brother.,E,possessive,her brother,water her garden at home,unrelated,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁Dorothy, ▁bike, ▁breaks, ▁down, ,...",0.023387,0.080523
796,"If Susan finishes her assignment early, her co...",Susan has a colleague.,E,possessive,his colleague,dust his shelves at the dorm,unrelated,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁Susan, ▁finishes, ▁her, ▁assignme...",0.022562,0.026473
797,"If Margaret finishes work early, her wife will...",Margaret has a wife.,E,possessive,his wife,plant a tree for him,unrelated,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁Margaret, ▁finishes, ▁work, ▁earl...",0.024631,NaN
798,"If Betty misses the train, her brother will ma...",Betty has a brother.,E,possessive,her brother,make her origami to class,unrelated,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁Betty, ▁misses, ▁the, ▁train, ,, ...",0.024545,0.086506


In [103]:
test_df5p['avg_key_phrase_attention'].isna().sum()

7

In [104]:
test_df5p['avg_attention'].mean()

0.022888529628980903

In [105]:
test_df5p['avg_key_phrase_attention'].mean()

0.0723602104213319

In [106]:
test_df5p.groupby('type')['avg_key_phrase_attention'].mean()

type
related      0.072417
unrelated    0.072303
Name: avg_key_phrase_attention, dtype: float64

In [107]:
test_df5p.to_csv('gemma_part5_type5p.csv')

## Type 5A

In [108]:
test_df5a

,premise,hypothesis,gold_label,trigger,key_phrase1,key_phrase2,type
0,"If Jessica attends a movie festival, she'll ne...",Jessica has watched a movie by Nolan before.,E,again,attends a movie festival,watch a movie by Nolan again,related
1,"If Emily finishes a mystery novel, she'll neve...",Emily has read a book by Dan Brown before.,E,again,finishes a mystery novel,read a book by Dan Brown again,related
2,"If Scarlett bakes a Swedish pastry, she'll nev...",Scarlett has eaten a traditional Swedish dish ...,E,again,bakes a Swedish pastry,eat a traditional Swedish dish again,related
3,"If Chloe attends a pop concert, she'll never s...",Chloe has sung a pop song before.,E,again,attends a pop concert,sing a pop song again,related
4,"If Charlotte attends a pop music festival, she...",Charlotte has listened to a pop song before.,E,again,attends a pop music festival,listen to a pop song again,related
...,...,...,...,...,...,...,...
795,"If Dorothy's car breaks down, she'll never obt...",Dorothy has obtained a job position before.,E,again,breaks down,obtain a job position again,unrelated
796,"If Susan finishes her work early, she'll never...",Susan has conveyed a confidential message before.,E,again,finishes her work early,convey a confidential message again,unrelated
797,"If Margaret finishes work early, she'll never ...",Margaret has managed to keep a secret before.,E,again,finishes work early,manage to keep a secret again,unrelated
798,"If Betty misses the bus, she'll never cherish ...",Betty has cherished a moment in peace before.,E,again,misses the bus,cherish a moment in peace again,unrelated


In [109]:
test_df5a[test_df5a['key_phrase2'].isna()]

,premise,hypothesis,gold_label,trigger,key_phrase1,key_phrase2,type


In [110]:
test_df5a[['attention','tokens']] = test_df5a.apply(average_attention,axis=1)
test_df5a[['avg_attention','avg_key_phrase_attention']] = test_df5a.apply(compute_attention_metrics, axis=1)
test_df5a

,premise,hypothesis,gold_label,trigger,key_phrase1,key_phrase2,type,attention,tokens,avg_attention,avg_key_phrase_attention
0,"If Jessica attends a movie festival, she'll ne...",Jessica has watched a movie by Nolan before.,E,again,attends a movie festival,watch a movie by Nolan again,related,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁Jessica, ▁attends, ▁a, ▁movie, ▁f...",0.018893,0.084949
1,"If Emily finishes a mystery novel, she'll neve...",Emily has read a book by Dan Brown before.,E,again,finishes a mystery novel,read a book by Dan Brown again,related,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁Emily, ▁finishes, ▁a, ▁mystery, ▁...",0.017763,0.085683
2,"If Scarlett bakes a Swedish pastry, she'll nev...",Scarlett has eaten a traditional Swedish dish ...,E,again,bakes a Swedish pastry,eat a traditional Swedish dish again,related,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁Scarlett, ▁b, akes, ▁a, ▁Swedish,...",0.017607,0.063260
3,"If Chloe attends a pop concert, she'll never s...",Chloe has sung a pop song before.,E,again,attends a pop concert,sing a pop song again,related,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁Chloe, ▁attends, ▁a, ▁pop, ▁conce...",0.019911,0.081718
4,"If Charlotte attends a pop music festival, she...",Charlotte has listened to a pop song before.,E,again,attends a pop music festival,listen to a pop song again,related,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁Charlotte, ▁attends, ▁a, ▁pop, ▁m...",0.018152,0.070429
...,...,...,...,...,...,...,...,...,...,...,...
795,"If Dorothy's car breaks down, she'll never obt...",Dorothy has obtained a job position before.,E,again,breaks down,obtain a job position again,unrelated,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁Dorothy, ', s, ▁car, ▁breaks, ▁do...",0.018357,0.097309
796,"If Susan finishes her work early, she'll never...",Susan has conveyed a confidential message before.,E,again,finishes her work early,convey a confidential message again,unrelated,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁Susan, ▁finishes, ▁her, ▁work, ▁e...",0.020318,0.083951
797,"If Margaret finishes work early, she'll never ...",Margaret has managed to keep a secret before.,E,again,finishes work early,manage to keep a secret again,unrelated,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁Margaret, ▁finishes, ▁work, ▁earl...",0.019484,0.109940
798,"If Betty misses the bus, she'll never cherish ...",Betty has cherished a moment in peace before.,E,again,misses the bus,cherish a moment in peace again,unrelated,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁Betty, ▁misses, ▁the, ▁bus, ,, ▁s...",0.019592,0.101446


In [111]:
test_df5a['avg_key_phrase_attention'].isna().sum()

2

In [112]:
test_df5a['avg_attention'].mean()

0.018549523575929924

In [113]:
test_df5a['avg_key_phrase_attention'].mean()

0.08302746441000536

In [114]:
test_df5a.groupby('type')['avg_attention'].mean()

type
related      0.018331
unrelated    0.018768
Name: avg_attention, dtype: float64

In [115]:
test_df5a.groupby('type')['avg_key_phrase_attention'].mean()

type
related      0.072538
unrelated    0.093517
Name: avg_key_phrase_attention, dtype: float64

In [116]:
test_df5a.to_csv('gemma_part5_type5a.csv')

## Attention from CLS token

In [26]:
def get_key_phrase_indices(row,attention_df):

    key_phrase1 = row['key_phrase1']
    key_phrase2 = row['key_phrase2']
    
    key_phrase1_words = key_phrase1.split()
    key_phrase2_words = key_phrase2.split()

    start_key_phrase1 = key_phrase1_words[0]
    end_key_phrase1 = key_phrase1_words[-1]
    start_key_phrase2 = key_phrase2_words[0]
    end_key_phrase2 = key_phrase2_words[-1]
    phrase_thresholds = [start_key_phrase1, end_key_phrase1, start_key_phrase2, end_key_phrase2]
    indices = {}
    try:
        indices['start_key_phrase1'] = list(attention_df.index).index(start_key_phrase1)
    except: 
        if 'his' in attention_df.index: 
            start_key_phrase1 = 'his'
        elif 'her' in attention_df.index: 
            start_key_phrase1 = 'her'
        elif 'their' in attention_df.index: 
            start_key_phrase1 = 'their'
        else:
            indices['start_key_phrase1'] = None
        indices['start_key_phrase1'] = list(attention_df.index).index(start_key_phrase1)

    try:
        indices['start_key_phrase2'] = list(attention_df.index).index(start_key_phrase2)
    except:
        if 'to' in attention_df.index: 
            indices['start_key_phrase2'] = list(attention_df.index).index('to') + 1
        else:
            indices['start_key_phrase2'] = None

    # searching for end index starts after start index
    try:
        indices['end_key_phrase1'] = indices['start_key_phrase1'] + list(attention_df.index)[indices['start_key_phrase1']:].index(end_key_phrase1) + 1
    except:
        if 'to' in attention_df.index: 
            indices['end_key_phrase1'] = list(attention_df.index).index('to')
        else:
            indices['end_key_phrase1'] = None 

    try:
        indices['end_key_phrase2'] = indices['start_key_phrase2'] + list(attention_df.index)[indices['start_key_phrase2']:].index(end_key_phrase2) + 1
    except:
        if '.' in attention_df.index: 
            indices['end_key_phrase2'] = list(attention_df.index).index('.')
        else:
            indices['end_key_phrase2'] = None 

    return indices

In [27]:
def gemma_key_attention_col(row):

    avg_attention = row['attention']
    tokens = row['tokens']
    attention_df = pd.DataFrame(avg_attention,index=tokens,columns=tokens)

    partial_words = [tok for i,tok in enumerate(attention_df.index) if not tok.startswith('▁') 
                and tok not in string.punctuation \
                and tok not in tokenizer.all_special_tokens \
                and attention_df.index[i-1] not in tokenizer.all_special_tokens]

    # if it's a partial word, add its row to the previous row (merging tokens)
    for i in range(len(attention_df)-1,0,-1):
        current_token = attention_df.index[i]
        prev_token = attention_df.index[i-1]
        if is_partial_word(current_token,prev_token):
            attention_df.iloc[i-1] += attention_df.iloc[i] # first row gets total value
            attention_df.iloc[i] = 0 # set second row to 0
            merged_token = attention_df.index.values[i-1] + attention_df.index.values[i]
            attention_df.index.values[i-1] = merged_token

    attention_df.index = [token[1:] if token.startswith('▁') else token for token in attention_df.index]

    # merging columns
    for i in range(attention_df.shape[1]-1,0,-1):
        current_token = attention_df.columns[i]
        prev_token = attention_df.columns[i-1]
        if is_partial_word(current_token,prev_token):
            attention_df.iloc[:, i-1] += attention_df.iloc[:, i]
            attention_df.iloc[:, i] = 0
            merged_token = attention_df.columns.values[i - 1] + current_token
            attention_df.columns.values[i-1] = merged_token

    # drop_cols = [col for col in attention_df.columns if col in partial_words]
    # attention_df = attention_df.drop(columns=drop_cols)
    attention_df.columns = [token[1:] if token.startswith('▁') else token for token in attention_df.columns ]
    cls_attention = attention_df.iloc[:,0] # first column is most important 
    avg_class_attention = cls_attention.mean()

    # compute average attention from key_phrase1 and key_phrase2 
    indices = get_key_phrase_indices(row,attention_df)

    if indices['start_key_phrase1'] and indices['end_key_phrase1']:
        key_phrase1_att = cls_attention.iloc[indices['start_key_phrase1']:indices['end_key_phrase1']]
        key_phrase1_att = key_phrase1_att.mean()
    else:
        key_phrase1_att = None 

    if indices['start_key_phrase2'] and indices['end_key_phrase2']:
        key_phrase2_att = cls_attention.iloc[indices['start_key_phrase2']:indices['end_key_phrase2']]
        key_phrase2_att = key_phrase2_att.mean()
    else:
        key_phrase2_att = None

    return pd.Series([cls_attention,avg_class_attention,key_phrase1_att,key_phrase2_att])

In [28]:
test_df4[['attention','tokens']] = test_df4.apply(average_attention,axis=1)
test_df4[['cls_attention','avg_cls_attention','key_phrase1_att','key_phrase2_att']] = test_df4.apply(gemma_key_attention_col,axis=1)
test_df4.to_csv('gemma_part5_type4_att_new.csv')
test_df4

,premise,hypothesis,gold_label,trigger,key_phrase1,key_phrase2,type,attention,tokens,cls_attention,avg_cls_attention,key_phrase1_att,key_phrase2_att
0,"If Liam is an aerospace engineer, he'll use hi...",Liam has a wind tunnel.,E,possessive,his wind tunnel,improve aircraft performance,related,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁Liam, ▁is, ▁an, ▁aerospace, ▁engi...",<bos> 1.000000 If 0.82313...,0.463922,0.459696,0.414086
1,"If Bill is a content strategist, he'll use his...",Bill has a content management application.,E,possessive,his content management application,plan and organize digital content,related,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁Bill, ▁is, ▁a, ▁content, ▁strateg...",<bos> 1.000000 If 0.82313...,0.441653,0.431097,0.393212
2,"If Steve is a conservation officer, he'll use ...",Steve has a wildlife trap.,E,possessive,his wildlife trap,monitor and protect animal populations,related,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁Steve, ▁is, ▁a, ▁conservation, ▁o...",<bos> 1.000000 If 0.823...,0.459281,0.455619,0.407110
3,"If Matt is a scuba diver, he'll wear his wetsu...",Matt has a wetsuit.,E,possessive,his wetsuit,maintain body temperature underwater,related,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁Matt, ▁is, ▁a, ▁scuba, ▁diver, ,,...",<bos> 1.000000 If 0.82313...,0.461310,0.655380,0.427398
4,"If Noah is a pastry chef, he'll use his pastry...",Noah has a pastry bag.,E,possessive,his pastry bag,decorate cakes and pastries,related,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁Noah, ▁is, ▁a, ▁pastry, ▁chef, ,,...",<bos> 1.000000 If 0.823137 Noah...,0.457270,0.436873,0.405209
...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,"If Christina is a makeup artist, she'll grab h...",Christina has a makeup sponge.,E,possessive,her makeup sponge,rides a scooter,unrelated,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁Christina, ▁is, ▁a, ▁makeup, ▁art...",<bos> 1.000000 If 0.823137 Ch...,0.458193,0.428663,0.435987
293,"If Rahim is a clergyman, he'll bring his holy ...",Rahim has a holy book.,E,possessive,his holy book,gardens roses,unrelated,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁Rahim, ▁is, ▁a, ▁clergyman, ,, ▁h...",<bos> 1.000000 If 0.823137 Ra...,0.448371,0.442978,0.421160
294,"If James is a police officer, he'll pick his g...",James has a gun.,E,possessive,his gun,drinks tea,unrelated,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁James, ▁is, ▁a, ▁police, ▁officer...",<bos> 1.000000 If 0.823137 James ...,0.477943,0.448389,0.450977
295,"If Maya is a piano teacher, she'll pack her me...",Maya has a metronome.,E,possessive,her metronome,knits hats,unrelated,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁Maya, ▁is, ▁a, ▁piano, ▁teacher, ...",<bos> 1.000000 If 0.823137 Ma...,0.462197,0.671826,0.439405


In [29]:
test_df5a[['attention','tokens']] = test_df5a.apply(average_attention,axis=1)
test_df5a[['cls_attention','avg_cls_attention','key_phrase1_att','key_phrase2_att']] = test_df5a.apply(gemma_key_attention_col,axis=1)
test_df5a.to_csv('gemma_part5_type5a_att_new.csv')
test_df5a

,premise,hypothesis,gold_label,trigger,key_phrase1,key_phrase2,type,attention,tokens,cls_attention,avg_cls_attention,key_phrase1_att,key_phrase2_att
0,"If Jessica attends a movie festival, she'll ne...",Jessica has watched a movie by Nolan before.,E,again,attends a movie festival,watch a movie by Nolan again,related,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁Jessica, ▁attends, ▁a, ▁movie, ▁f...",<bos> 1.000000 If 0.823137 Jess...,0.430916,0.541545,0.423830
1,"If Emily finishes a mystery novel, she'll neve...",Emily has read a book by Dan Brown before.,E,again,finishes a mystery novel,read a book by Dan Brown again,related,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁Emily, ▁finishes, ▁a, ▁mystery, ▁...",<bos> 1.000000 If 0.823137 Emil...,0.422728,0.536702,0.427125
2,"If Scarlett bakes a Swedish pastry, she'll nev...",Scarlett has eaten a traditional Swedish dish ...,E,again,bakes a Swedish pastry,eat a traditional Swedish dish again,related,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁Scarlett, ▁b, akes, ▁a, ▁Swedish,...",<bos> 1.000000 If 0.82313...,0.426921,0.552219,0.417878
3,"If Chloe attends a pop concert, she'll never s...",Chloe has sung a pop song before.,E,again,attends a pop concert,sing a pop song again,related,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁Chloe, ▁attends, ▁a, ▁pop, ▁conce...",<bos> 1.000000 If 0.823137 Chloe ...,0.443952,0.552961,0.425971
4,"If Charlotte attends a pop music festival, she...",Charlotte has listened to a pop song before.,E,again,attends a pop music festival,listen to a pop song again,related,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁Charlotte, ▁attends, ▁a, ▁pop, ▁m...",<bos> 1.000000 If 0.823137 Ch...,0.433767,0.540439,0.415574
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,"If Dorothy's car breaks down, she'll never obt...",Dorothy has obtained a job position before.,E,again,breaks down,obtain a job position again,unrelated,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁Dorothy, ', s, ▁car, ▁breaks, ▁do...",<bos> 1.000000 If 0.823137 Do...,0.441642,0.504173,0.443094
796,"If Susan finishes her work early, she'll never...",Susan has conveyed a confidential message before.,E,again,finishes her work early,convey a confidential message again,unrelated,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁Susan, ▁finishes, ▁her, ▁work, ▁e...",<bos> 1.000000 If 0.823...,0.437554,0.534937,0.437412
797,"If Margaret finishes work early, she'll never ...",Margaret has managed to keep a secret before.,E,again,finishes work early,manage to keep a secret again,unrelated,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁Margaret, ▁finishes, ▁work, ▁earl...",<bos> 1.000000 If 0.823137 Marg...,0.431520,0.541341,0.445542
798,"If Betty misses the bus, she'll never cherish ...",Betty has cherished a moment in peace before.,E,again,misses the bus,cherish a moment in peace again,unrelated,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁Betty, ▁misses, ▁the, ▁bus, ,, ▁s...",<bos> 1.000000 If 0.823137 Be...,0.429143,0.551529,0.440774


In [30]:
test_df5p[['attention','tokens']] = test_df5p.apply(average_attention,axis=1)
test_df5p[['cls_attention','avg_cls_attention','key_phrase1_att','key_phrase2_att']] = test_df5p.apply(gemma_key_attention_col,axis=1)
test_df5p.to_csv('gemma_part5_type5p_att_new.csv')
test_df5p

,premise,hypothesis,gold_label,trigger,key_phrase1,key_phrase2,type,attention,tokens,cls_attention,avg_cls_attention,key_phrase1_att,key_phrase2_att
0,"If Jessica attends the conference, her colleag...",Jessica has a colleague.,E,possessive,her colleague,the university,related,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁Jessica, ▁attends, ▁the, ▁confere...",<bos> 1.000000 If 0.823137 ...,0.460026,0.468288,0.448668
1,"If Emily passes her driving test, her best fri...",Emily has a best friend.,E,possessive,her best friend,post exam celebratory dinner,related,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁Emily, ▁passes, ▁her, ▁driving, ▁...",<bos> 1.000000 If 0.82313...,0.456283,0.499912,0.421428
2,"If Scarlett finishes her chores early, her hus...",Scarlett has a husband.,E,possessive,her husband,lunch,related,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁Scarlett, ▁finishes, ▁her, ▁chore...",<bos> 1.000000 If 0.823137 Scar...,0.462027,0.505512,0.434286
3,"If Chloe finishes his project early, his colle...",Chloe has colleagues.,E,possessive,his colleagues,drinks,related,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁Chloe, ▁finishes, ▁his, ▁project,...",<bos> 1.000000 If 0.823137 ...,0.474208,0.485197,0.437529
4,"If Charlotte passes her math test, her parents...",Charlotte has parents.,E,possessive,her parents,a calculator,related,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁Charlotte, ▁passes, ▁her, ▁math, ...",<bos> 1.000000 If 0.823137 ...,0.485704,0.503435,0.440595
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,"If Dorothy bike breaks down, her brother will ...",Dorothy has a brother.,E,possessive,her brother,water her garden at home,unrelated,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁Dorothy, ▁bike, ▁breaks, ▁down, ,...",<bos> 1.000000 If 0.823137 Doroth...,0.456965,0.489312,0.426741
796,"If Susan finishes her assignment early, her co...",Susan has a colleague.,E,possessive,his colleague,dust his shelves at the dorm,unrelated,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁Susan, ▁finishes, ▁her, ▁assignme...",<bos> 1.000000 If 0.823137 ...,0.465351,0.357340,0.429457
797,"If Margaret finishes work early, her wife will...",Margaret has a wife.,E,possessive,his wife,plant a tree for him,unrelated,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁Margaret, ▁finishes, ▁work, ▁earl...",<bos> 1.000000 If 0.823137 Marg...,0.464535,0.481027,0.416842
798,"If Betty misses the train, her brother will ma...",Betty has a brother.,E,possessive,her brother,make her origami to class,unrelated,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[<bos>, If, ▁Betty, ▁misses, ▁the, ▁train, ,, ...",<bos> 1.000000 If 0.823137 Betty ...,0.465646,0.476651,0.420380
